In [1]:
# procesamiento.py
import os
import re
import fitz  # PyMuPDF
import spacy

# Cargar modelo de lenguaje para NLP
nlp = spacy.load("en_core_web_sm")

def extraer_autores_de_pdf(ruta_pdf):
    """Extrae los nombres de autores de un PDF académico."""
    doc = fitz.open(ruta_pdf)
    texto = ""
    
    # Extraer texto de las primeras páginas (donde suelen estar los autores)
    for pagina in doc[:2]:
        texto += pagina.get_text()
    
    # Buscar patrones comunes de secciones de autores
    patrones = [
        r"Authors?:\s*(.+)",
        r"By\s*(.+)",
        r"^([A-Z][a-z]+ [A-Z][a-z]+(?:, | and )?)+",
    ]
    
    autores = []
    for patron in patrones:
        coincidencias = re.search(patron, texto, re.MULTILINE)
        if coincidencias:
            texto_autores = coincidencias.group(1)
            # Procesar con NLP para identificar nombres propios
            doc_nlp = nlp(texto_autores)
            autores = [ent.text for ent in doc_nlp.ents if ent.label_ == "PERSON"]
            if autores:
                break
    
    return tuple(sorted(set(autores)))  # Elimina duplicados y ordena

def procesar_carpeta(carpeta):
    """Procesa todos los PDFs en una carpeta y devuelve las tuplas de autores."""
    colaboraciones = []
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".pdf"):
            ruta = os.path.join(carpeta, archivo)
            autores = extraer_autores_de_pdf(ruta)
            if autores:
                colaboraciones.append(autores)
                print(f"Procesado {archivo}: {autores}")
    return colaboraciones

if __name__ == "__main__":
    articulos_dir = "articulos"
    colaboraciones = procesar_carpeta(articulos_dir)
    print("\nResumen de colaboraciones:")
    for i, auts in enumerate(colaboraciones, 1):
        print(f"Artículo {i}: {auts}")

ModuleNotFoundError: No module named 'fitz'